# ETL for Fake Reviews
_____

In [1]:
import numpy as np
import pandas as pd

In [2]:
file = 'source/fakespot_fake_rating_f.txt'
with open(file) as f:
    content = f.read()

In [3]:
content = content.split('Report')
print(content[0])

	Charles
5.0 out of 5 stars A17 great bang for your buck. And waterprooof.
Reviewed in the United States on July 23, 2023
Verified Purchase
They are great ear buds they are loud and the music is always crisp and clear. They fit in my ear just great and the ear hooks are what I need because air pods fall out on me. As for the microphone function it is great and I can hear the people I’m talking to and they can hear me and doesn’t sound like a tin can and eliminates much of the background noise. It has a long battery life with double led displays showing all battery strengths. I would recommend getting another pair for my family. Since my purchase I get in the car in the morning and the phone pairs to the car's audio system. When I get to work, I park the car, reach into my bag and take out either the left or the right bud, and put it in my ear. It turns on and connects immediately, with audible notifications, and the music automatically switches from the car to the bud without any butto

In [4]:
content2 = []
for c in content:
    content2.append([i.strip() for i in c.split('\n') if i.strip()])
content2 = [c for c in content2 if len(c) > 0]
len(content2)

435

In [16]:
[c[1] for c in content2]

['5.0 out of 5 stars A17 great bang for your buck. And waterprooof.',
 '5.0 out of 5 stars Simple to pair to phone and High quality sound',
 '5.0 out of 5 stars Seamless connection to smartphone',
 '5.0 out of 5 stars Comfortable, great sounding headphones that actually stay in my ears',
 '5.0 out of 5 stars Gold Standard',
 '5.0 out of 5 stars Great sound and so easy to setup',
 '5.0 out of 5 stars Good value for price',
 '5.0 out of 5 stars Clear voice quality and great fit - No fall',
 '5.0 out of 5 stars Excellent Value',
 '5.0 out of 5 stars Durable and Intuitive Earbuds - Great for Exercising!',
 '5.0 out of 5 stars Really nice with out a big price.',
 '5.0 out of 5 stars Boloxa A17 Bluetooth Earbuds Review - Your Buds for Sound and Comfort!',
 '5.0 out of 5 stars Quality and comfort at an affordable price',
 '5.0 out of 5 stars Very comfortable, all-purpose earbuds',
 '5.0 out of 5 stars A Symphony for Your Ears - BOLOXA A17 Bluetooth Headphones!',
 '5.0 out of 5 stars Very easy

In [5]:
# these are used instead of names
special = [
'Translate review to English',
'Muy buen producto el pedido llego increíble tiene todo completo',
'Received my package fast and they play loud n clear havent had any problems out of them i didn’t have to buy new air pods after all these are just fine good quality',
]

# modify content2 to deal w/items from var special
idx_delete = []
idx_modify = []
for idx, c in enumerate(content2):
    # remove
    if special[0] in c:
        content2[idx] = [item for item in c if item != special[0]]
    # to list for removal
    if special[1] in c:
        idx_delete.append(idx)
    # to list for modification
    if special[2] in c:
        idx_modify.append(idx)
print(idx_delete)
print(idx_modify)
print([i for i in content2 if special[0] in i])

[242]
[235]
[]


In [6]:
del content2[242]
content2[235][0] = 'Uknown'

In [7]:
# remove 'VINE VOICE' from content2, but keep this info in a separate data structure
vine_voices = [False]*len(content2)
line = 'VINE VOICE'
for idx, c in enumerate(content2):
    if line in c:
        content2[idx] = [item for item in c if item != line]
        vine_voices[idx] = True
sum(vine_voices)

42

In [8]:
# remove images, but keep info
images = [
    'Customer image',
    'Customer imageCustomer image',
    'Customer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer image',
]
images_available = [False]*len(content2)
for image in images:
    for idx, c in enumerate(content2):
        if image in c:
            content2[idx] = [item for item in c if item != image]
            images_available[idx] = True
sum(images_available)

136

In [9]:
# remove 'found helpful', but keep info
helpful = [ 'One person found this helpful', ] + [str(i) + ' people found this helpful' for i in range(2,501)]
found_helpful = [0]*len(content2)
for h in helpful:
    for idx, c in enumerate(content2):
        if h in c:
            content2[idx] = [item for item in c if item != h]
            num = h.split()[0].lower().strip()
            found_helpful[idx] = 1 if num=='one' else int(num)
len([i for i in found_helpful if i>0])

55

In [10]:
# remove 'Verified Purchase', but keep info
line = 'Verified Purchase'
verified_purchase = [False]*len(content2)
for idx, c in enumerate(content2):
        if line in c:
            content2[idx] = [item for item in c if item != line]
            verified_purchase[idx] = True
print(sum(verified_purchase))

269


In [11]:
# this info is already available, just remove
line = "Vine Customer Review of Free Product( What's this? )"
count = 0
for idx, c in enumerate(content2):
        if line in c:
            content2[idx] = [item for item in c if item != line]
            count += 1
print(f"Removed this {count} times")

Removed this 50 times


__PREPROCESSING SUMMARY__
* vine_voices keep vine voices
* images_available keep info about images
* found_helpful - contains info about # people who found this helpful
* verified_purchase - info about %

* i[0] - name
* i[1] - stars + title `'5.0 out of 5 stars Work great and sound great',`
* i[2] - date `'Reviewed in the United States on July 25, 2023'`
* i[3] - color or Verified Purchaser or both or vine (fix in the df): `'Color: BLACKVerified Purchase'`,
 `'Color: WhiteVerified Purchase'`, `'Color: Black'`, `'Color: White'`
* i[-1] - Helpful

In [18]:
set([len(i) for i in content2])

{5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 23}

In [19]:
# get text by excluding other columns (-1 for Helpful)
text = ['\n'.join(i[3:-1]) for i in content2]
len(text)

434

In [20]:
df_fake = pd.DataFrame({
    'text': text,
    'name': [i[0] for i in content2],
    'stars_title': [i[1] for i in content2],
    'date': [i[2] for i in content2],
    'is_vine_voice': vine_voices,
    'has_images': images_available,
    'found_helpful': found_helpful,
    'verified_purchase': verified_purchase,
})
print(df_fake.shape)
df_fake.head()

(434, 8)


,text,name,stars_title,date,is_vine_voice,has_images,found_helpful,verified_purchase
0,They are great ear buds they are loud and the ...,Charles,5.0 out of 5 stars A17 great bang for your buc...,"Reviewed in the United States on July 23, 2023",False,True,16,True
1,This is a review for the Boloxa wireless headp...,Cloudy,5.0 out of 5 stars Simple to pair to phone and...,"Reviewed in the United States on August 23, 2023",False,False,5,True
2,I've tried plenty of in-ear or wrap-around hea...,Timmy Teplant,5.0 out of 5 stars Seamless connection to smar...,"Reviewed in the United States on August 7, 2023",False,False,3,True
3,After buying various in-ear headphones for wal...,Ryan,"5.0 out of 5 stars Comfortable, great sounding...","Reviewed in the United States on July 20, 2023",False,False,1,True
4,These ear buds are tied with only one other ch...,Patricia,5.0 out of 5 stars Gold Standard,"Reviewed in the United States on July 31, 2023",False,False,2,True


In [21]:
# get date, convert to datetime
line = 'Reviewed in the United States on'
df_fake['date'] = df_fake['date'].apply(lambda x: x.replace(line,'').strip())
df_fake['date'] = df_fake['date'].str.strip()
df_fake['date'] = pd.to_datetime( df_fake['date'] )

In [22]:
# split stars and title into separate columns
df_fake[['stars', 'title']] = df_fake['stars_title'].str.split('out of 5 stars', 2, expand=True)
df_fake['stars'] = df_fake['stars'].str.strip()
df_fake['stars'] = df_fake['stars'].astype(float)
df_fake['stars'] = df_fake['stars'].apply(lambda x: int(x))
df_fake['title'] = df_fake['title'].str.strip()
df_fake = df_fake.drop(['stars_title'], axis=1)
df_fake['text'] = df_fake['text'].str.strip()

/var/folders/vb/p2r9brhx2335cwnww04p9w180000gn/T/ipykernel_48601/893117490.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_fake[['stars', 'title']] = df_fake['stars_title'].str.split('out of 5 stars', 2, expand=True)


In [23]:
df_fake.head()

,text,name,date,is_vine_voice,has_images,found_helpful,verified_purchase,stars,title
0,They are great ear buds they are loud and the ...,Charles,2023-07-23,False,True,16,True,5,A17 great bang for your buck. And waterprooof.
1,This is a review for the Boloxa wireless headp...,Cloudy,2023-08-23,False,False,5,True,5,Simple to pair to phone and High quality sound
2,I've tried plenty of in-ear or wrap-around hea...,Timmy Teplant,2023-08-07,False,False,3,True,5,Seamless connection to smartphone
3,After buying various in-ear headphones for wal...,Ryan,2023-07-20,False,False,1,True,5,"Comfortable, great sounding headphones that ac..."
4,These ear buds are tied with only one other ch...,Patricia,2023-07-31,False,False,2,True,5,Gold Standard


In [24]:
# reorder columns
cols = [ 'title', 'text', 'name', 'date', 'stars', 'is_vine_voice', 'has_images', 'found_helpful',
         'verified_purchase',  ]
df_fake = df_fake[cols]
df_fake.head()

,title,text,name,date,stars,is_vine_voice,has_images,found_helpful,verified_purchase
0,A17 great bang for your buck. And waterprooof.,They are great ear buds they are loud and the ...,Charles,2023-07-23,5,False,True,16,True
1,Simple to pair to phone and High quality sound,This is a review for the Boloxa wireless headp...,Cloudy,2023-08-23,5,False,False,5,True
2,Seamless connection to smartphone,I've tried plenty of in-ear or wrap-around hea...,Timmy Teplant,2023-08-07,5,False,False,3,True
3,"Comfortable, great sounding headphones that ac...",After buying various in-ear headphones for wal...,Ryan,2023-07-20,5,False,False,1,True
4,Gold Standard,These ear buds are tied with only one other ch...,Patricia,2023-07-31,5,False,False,2,True


In [25]:
# extract colors and more data on verified purchases
colors = ['Color: BLACKVerified Purchase', 'Color: WhiteVerified Purchase', 'Color: Black', 'Color: White',]
indirect_colors = ['black color', 'red color', ]
df_fake['color'] = np.nan

def exctract_color(row):
    t = row['text']
    if colors[0] in t:
        row['text'] = t.replace(colors[0], '').strip()
        row['color'] = 'black'
        row['verified_purchase'] = True
    elif colors[1] in t:
        row['text'] = t.replace(colors[1], '').strip()
        row['color'] = 'white'
        row['verified_purchase'] = True
    elif colors[2] in t:
        row['text'] = t.replace(colors[2], '').strip()
        row['color'] = 'black'
    elif colors[3] in t:
        row['text'] = t.replace(colors[3], '').strip()
        row['color'] = 'white'
    elif indirect_colors[0] in t:
        row['color'] = 'black'
    elif indirect_colors[1] in t:
        row['color'] = 'red'
    return row

df_fake = df_fake.apply( exctract_color, axis=1 )
df_fake['color'].value_counts()

white    77
black    38
red       2
Name: color, dtype: int64

In [26]:
# reorder columns
cols = [ 'title', 'text', 'name', 'date', 'stars', 'color', 'is_vine_voice', 'has_images', 'found_helpful',
         'verified_purchase',  ]
df_fake = df_fake[cols]
df_fake.head()

,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase
0,A17 great bang for your buck. And waterprooof.,They are great ear buds they are loud and the ...,Charles,2023-07-23,5,NaN,False,True,16,True
1,Simple to pair to phone and High quality sound,This is a review for the Boloxa wireless headp...,Cloudy,2023-08-23,5,NaN,False,False,5,True
2,Seamless connection to smartphone,I've tried plenty of in-ear or wrap-around hea...,Timmy Teplant,2023-08-07,5,black,False,False,3,True
3,"Comfortable, great sounding headphones that ac...",After buying various in-ear headphones for wal...,Ryan,2023-07-20,5,NaN,False,False,1,True
4,Gold Standard,These ear buds are tied with only one other ch...,Patricia,2023-07-31,5,NaN,False,False,2,True


In [28]:
# check if empty text - there is title for 285
df_fake[ df_fake['text']=='' ]

,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase
285,Wonderful,,Samantha,2023-08-31,5,white,False,True,0,True


In [29]:
df_fake.loc[285, 'text'] = 'Wonderful'

In [31]:
# text duplicates, but other elements in row are different
# 271 and 370 are not really duplicates
df_fake.drop_duplicates()[ df_fake.drop_duplicates().duplicated(subset=['text'], keep=False) ].sort_values(by='text')

,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase
271,Quality,Good product,Good product,2023-09-08,5,black,False,False,0,True
370,Good product,Good product,pigchop,2023-08-22,5,NaN,False,False,0,True
11,Boloxa A17 Bluetooth Earbuds Review - Your Bud...,I just got my hands on the Boloxa A17 Bluetoot...,Mayy,2023-08-09,5,NaN,False,False,0,True
100,Boloxa A17 Bluetooth Earbuds Review - Your Bud...,I just got my hands on the Boloxa A17 Bluetoot...,Mayy,2023-08-09,5,NaN,False,False,1,True
2,Seamless connection to smartphone,I've tried plenty of in-ear or wrap-around hea...,Timmy Teplant,2023-08-07,5,black,False,False,3,True
92,Seamless connection to smartphone,I've tried plenty of in-ear or wrap-around hea...,Timmy Teplant,2023-08-07,5,black,False,False,5,True
4,Gold Standard,These ear buds are tied with only one other ch...,Patricia,2023-07-31,5,NaN,False,False,2,True
93,Gold Standard,These ear buds are tied with only one other ch...,Patricia,2023-07-31,5,NaN,False,False,4,True


In [32]:
# drop special-case duplicates
df_fake = df_fake.drop([2, 4, 11,])
df_fake.shape

(431, 10)

In [33]:
# drop duplicates
df_fake = df_fake.drop_duplicates()
df_fake.shape

(356, 10)

In [34]:
# filter out non-English reviews and replace emojis with text
import pycld2 as cld2
import emoji

def detect_lang( s ):
    '''
        Return the language(s) in string s.
        Naive Bayes classifier under the hood -
        results are less certain for strings that are too short.
        Returns up to three languages with confidence scores.
        More on usage: https://pypi.org/project/pycld2/
    '''
    _, _, details = cld2.detect( s )
    return details[0][0]


def replace_emoji(s, to_text=True):
    '''
        Replace emojis in string s with their corresponding textual description if to_text=True;
        otherwise, delete all emojis
    '''

    # convert emojis to text
    if to_text:
        s2 = emoji.demojize( s )

    # or delete emojis
    else:
        for e in emoji.EMOJI_DATA:
            if e in s:
                s = s.replace(e, '')
    # see what's changed    
    if s2 != s:
        print('OLD TEXT:\n', s, sep='')
        print('\nNEW TEXT:\n', s2, sep='')
        print('\n', '='*75, '\n', sep='')

    return s2

In [35]:
df_fake['lang'] = df_fake['text'].apply(detect_lang)
df_fake['lang'].value_counts()

ENGLISH    347
Unknown      7
SPANISH      2
Name: lang, dtype: int64

In [37]:
# these look English
df_fake[ df_fake['lang']=='Unknown' ]

,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase,lang
269,They were a gift,They work fine.,Jennifer Abate,2023-09-06,5,black,False,False,0,True,Unknown
271,Quality,Good product,Good product,2023-09-08,5,black,False,False,0,True,Unknown
280,Good,Good,Sonya Godfrey,2023-09-03,5,black,False,False,0,True,Unknown
284,Works great,Love them,Angela M. Mcbroom,2023-08-31,5,white,False,False,0,True,Unknown
285,Wonderful,Wonderful,Samantha,2023-08-31,5,white,False,True,0,True,Unknown
370,Good product,Good product,pigchop,2023-08-22,5,NaN,False,False,0,True,Unknown
375,Great headphones!,Excellent value.,Cameron Ruppe,2023-08-15,5,NaN,False,False,0,True,Unknown


In [38]:
# SPANISH alright
df_fake[ df_fake['lang']=='SPANISH' ]

,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase,lang
62,"Buen sonido, buen micrófono, no se caen con el...","Practicar deportes es más cómodo con música, p...",Orlando Rodriguez,2023-08-15,5,NaN,False,False,0,True,SPANISH
238,Audífonos,Muy buena entrega y muy buena la calidad de envío,Jorge Morales,2023-09-07,5,black,False,False,0,True,SPANISH


In [40]:
# remove SPANISH
df_fake = df_fake[ df_fake['lang']!='SPANISH' ]
df_fake.shape

(354, 11)

In [41]:
# replace emoji
df_fake['text'] = df_fake['text'].apply(replace_emoji)

OLD TEXT:
Well my earbuds showed up today which I’m super stoked about because the earbuds I am currently using while working fit in my ears but they aren’t snug.. so I ordered these in hopes they will fulfill my issue I’m currently having.
I have to say that these earbuds immediately paired with my iPhone 14 😁 after having them paired I quickly put these on my ears and they slid right on and then in went the ear piece and you know the feeling when you have an exact fit 🥳 🤠 I knew off the muscle these are gonna be the ones that do me right… after doing this and having my extreme feeling of satisfaction I went to my pandora app where my music kicked on out went all the outside noise and in went some kick ass music.. I have to say I truly think these are gonna be the best buds I’ve ever owned..
I went through all the prompts that the instructions manual informed me of.. one push to stop the music one push to stop the music then a punch and hold to turn the volume up and down
Considering 

In [42]:
df_fake['title'] = df_fake['title'].apply(replace_emoji)

OLD TEXT:
Amazing quality 👌

NEW TEXT:
Amazing quality :OK_hand:


OLD TEXT:
Very good quality 👌

NEW TEXT:
Very good quality :OK_hand:


OLD TEXT:
I ❤️ these things!!😎🌴

NEW TEXT:
I :red_heart: these things!!:smiling_face_with_sunglasses::palm_tree:




In [43]:
# save to pickle
file = 'source/df_fake.pkl'
df_fake.to_pickle(file)